In [2]:
import numpy as np
import pandas as pd
import re

In [3]:
delitos = pd.read_csv('https://www.datos.gov.co/api/views/75fz-q98y/rows.csv?accessType=DOWNLOAD&bom=true&format=true')

/Users/giovannydelacruz/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
delitos

,ORDEN,ARMAS_MEDIOS,BARRIOS_HECHO,LATITUD,LONGITUD,ZONA,NOM_COMUNA,ANO,MES,DIA,DIA_SEMANA,DESCRIPCION_CONDUCTA,CONDUCTA,CLASIFICACIONES DELITO,EDAD,CURSO_DE_VIDA,ESTADO_CIVIL_PERSONA,GENERO,MOVIL_AGRESOR,MOVIL_VICTIMA
0,1,ARMA BLANCA / CORTOPUNZANTE,BUENOS AIRES,7.170557,-73.135108,URBANA,14. Morrorico,2010,01. Enero,1,05. Viernes,ARTÍCULO 111. LESIONES PERSONALES,LESIONES PERSONALES,Lesiones no fatales,30,05. Adultez,UNION LIBRE,MASCULINO,A PIE,A PIE
1,2,ARMA BLANCA / CORTOPUNZANTE,CAMPO HERMOSO,7.120645,-73.12605,URBANA,05. García Rovira,2010,01. Enero,1,05. Viernes,ARTÍCULO 111. LESIONES PERSONALES,LESIONES PERSONALES,Lesiones no fatales,21,04. Jovenes,SOLTERO,MASCULINO,A PIE,A PIE
2,3,ARMA BLANCA / CORTOPUNZANTE,CAMPO HERMOSO,7.120645,-73.12605,URBANA,05. García Rovira,2010,01. Enero,1,05. Viernes,ARTÍCULO 111. LESIONES PERSONALES,LESIONES PERSONALES,Lesiones no fatales,23,04. Jovenes,SOLTERO,MASCULINO,A PIE,A PIE
3,4,ARMA BLANCA / CORTOPUNZANTE,COMUNEROS,7.151359,-73.145705,URBANA,03. San Francisco,2010,01. Enero,1,05. Viernes,ARTÍCULO 111. LESIONES PERSONALES,LESIONES PERSONALES,Lesiones no fatales,36,05. Adultez,CASADO,MASCULINO,A PIE,A PIE
4,5,ARMA BLANCA / CORTOPUNZANTE,GIRARDOT,7.170557,-73.135108,URBANA,04. Occidental,2010,01. Enero,1,05. Viernes,ARTÍCULO 111. LESIONES PERSONALES,LESIONES PERSONALES,Lesiones no fatales,20,04. Jovenes,UNION LIBRE,MASCULINO,A PIE,A PIE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135071,135072,SIN EMPLEO DE ARMAS,CABECERA DEL LLANO,NaN,NaN,NO REPORTA,12. Cabecera del Llano,2021,12. Diciembre,31,05. Viernes,ARTÍCULO 239. HURTO A PERSONAS,HURTO A PERSONAS,Lesiones no fatales,25,04. Jovenes,NO REPORTA,NO REPORTA,A PIE,PASAJERO MOTOCICLETA
135072,135073,ARMA DE FUEGO,JARDINES DE COAVICONSA,NaN,NaN,NO REPORTA,11. Sur,2021,12. Diciembre,31,05. Viernes,ARTÍCULO 239. HURTO A PERSONAS,HURTO A PERSONAS,Lesiones no fatales,21,04. Jovenes,NO REPORTA,NO REPORTA,A PIE,CONDUCTOR MOTOCICLETA
135073,135074,ARMA DE FUEGO,PROVENZA,NaN,NaN,NO REPORTA,10. Provenza,2021,12. Diciembre,31,05. Viernes,ARTÍCULO 239. HURTO A PERSONAS,HURTO A PERSONAS,Lesiones no fatales,25,04. Jovenes,NO REPORTA,NO REPORTA,A PIE,CONDUCTOR MOTOCICLETA
135074,135075,ARMA BLANCA / CORTOPUNZANTE,TOLEDO PLATA,NaN,NaN,NO REPORTA,11. Sur,2021,12. Diciembre,31,05. Viernes,ARTÍCULO 239. HURTO A PERSONAS,HURTO A PERSONAS,Lesiones no fatales,47,05. Adultez,NO REPORTA,NO REPORTA,A PIE,A PIE


In [5]:
delitos.columns= delitos.columns.str.lower()

In [6]:
delitos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135076 entries, 0 to 135075
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   orden                   135076 non-null  int64 
 1   armas_medios            135076 non-null  object
 2   barrios_hecho           135076 non-null  object
 3   latitud                 128713 non-null  object
 4   longitud                128713 non-null  object
 5   zona                    135076 non-null  object
 6   nom_comuna              135076 non-null  object
 7   ano                     135076 non-null  int64 
 8   mes                     135076 non-null  object
 9   dia                     135076 non-null  int64 
 10  dia_semana              135076 non-null  object
 11  descripcion_conducta    135076 non-null  object
 12  conducta                135076 non-null  object
 13  clasificaciones delito  135076 non-null  object
 14  edad                    135076 non-n

In [7]:
delitos['nom_comuna'] = delitos['nom_comuna'].str.upper()
delitos['nom_comuna'] = delitos['nom_comuna'].str.replace('.',' -')

/var/folders/s2/7zt_5m2d2nv76vq_ybqf9csm0000gn/T/ipykernel_7646/1842341404.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  delitos['nom_comuna'] = delitos['nom_comuna'].str.replace('.',' -')


In [8]:
delitos['ano'] = pd.to_datetime(delitos['ano'], format="%Y").dt.year

In [9]:
delitos["mes"] = delitos['mes'].str[0:2].astype(int)

In [10]:
order = '[^0-9]'
dia= list(delitos['dia_semana'].str.lower())
dias_clean = []
for i in range(len(dia)):
    
    filtered_string = re.sub(order,'',dia[i])
    dias_clean.append(filtered_string.lstrip())        

dias_clean = pd.Series(dias_clean)

In [11]:
delitos['dia_semana'] = pd.to_datetime(dias_clean, format="%d").dt.day

In [12]:
delitos["armas_medios"].replace("NO REPORTADO",np.nan, inplace = True)
delitos["barrios_hecho"].replace("sin informacion",np.nan, inplace = True)
delitos["zona"].replace("NO REPORTA",np.nan, inplace = True)
delitos["nom_comuna"].replace("SIN INFORMACION",np.nan, inplace = True)
delitos["descripcion_conducta"].replace("NO REPORTA",np.nan, inplace = True)
delitos["conducta"].replace("NO REPORTA",np.nan, inplace = True)
delitos["clasificaciones delito"].replace("NO REPORTA",np.nan, inplace = True)
delitos["edad"].replace("NO REPORTA",np.nan, inplace = True)
delitos["curso_de_vida"].replace(["NO REPORTA","ERROR: #N/A"],np.nan, inplace = True)
delitos["estado_civil_persona"].replace("NO REPORTA",np.nan, inplace = True)
delitos["genero"].replace("NO REPORTA",np.nan, inplace = True)
delitos["movil_agresor"].replace("NO REPORTA",np.nan, inplace = True)
delitos["movil_victima"].replace("NO REPORTA",np.nan, inplace = True)

In [13]:
delitos["barrios_hecho"] =delitos["barrios_hecho"].str.upper()

Se pasara a filtrar y dejar unicamente los barrios de la alerta temprana.

In [14]:
delitos["barrios_hecho"].sort_values().unique()

array(['12 DE OCTUBRE', '13 DE JUNIO', '20 DE JULIO', '23 DE JUNIO',
       '5 DE ENERO', 'ACACIAS', 'AFRICA', 'ALARCON', 'ALBANIA',
       'ALFONSO LOPEZ', 'ALTO VIENTO', 'ALTO VIENTO I', 'ALTO VIENTO II',
       'ALTOS DE CAÑAVERAL CAMPESTRE', 'ALTOS DE CAÑAVERAL I',
       'ALTOS DEL CACIQUE', 'ALTOS DEL CAMPESTRE', 'ALTOS DEL KENNEDY',
       'ALTOS DEL PROGRESO', 'ALVAREZ', 'ALVAREZ LAS AMERICAS',
       'ANDALUCIA', 'ANILLO VIAL', 'ANTIGUA COLOMBIA',
       'ANTIGUO CAMPESTRE', 'ANTIGUO COLOMBIA', 'ANTONIA SANTOS',
       'ANTONIA SANTOS CENTRO', 'ANTONIA SANTOS SUR',
       'ASENTAMIENTO SERVIUNON', 'ASENTAMIENTO SERVUNION', 'ASTURIAS',
       'AUTOPISTA F/BLANCA-P/CUESTA.', 'AUTOPISTA P/CUESTA - F/BLANCA.',
       'BALCON DEL LAGO', 'BALCON DEL TEJAR', 'BALCONCITOS',
       'BALCONES DEL KENNEDY', 'BALCONES DEL SUR', 'BARRIO NUEVO',
       'BAVARIA', 'BELLAVISTA', 'BELLAVISTA CASAS', 'BELLAVISTA FLORIDA',
       'BETANIA', 'BOLARQUI', 'BOLIVAR', 'BONANZA CAMPESTRE (COLORADOS)',

In [15]:
barrios_alertatemprana = ["12 DE OCTUBRE","13 DE JUNIO","23 DE JUNIO","5 DE ENERO","ALARCON","BETANIA","ALTOS DEL KENNEDY","ALTOS DEL PROGRESO","BALCONES DEL KENNEDY",
                        "BARRIO NUEVO","BOSQUE NORTE","BUENOS AIRES","CAFE MADRID","CAMILO TORRES","CAMPO MADRID","CARLOS PIZARRO","CHAPINERO","CLAVERIANO","COLORADOS",
                        "COMUNEROS","DIVINO NIÑO","DON BOSCO","EL CINAL","EL ROSAL","EL PABLÓN","LA ESPERANZA I","LA ESPERANZA II", "LA ESPERANZA III","ESTORAQUES",
                        "GAITAN","GALLINERAL","GIRARDOT","GRANADA","JOSE ANTONIO GALAN","JOSE MARIA CORDOBA","KENNEDY","LA FERIA","LA GLORIA","LA INDEPENDENCIA",
                        "LA JUVENTUD","LA INMACULADA","LA UNIVERSIDAD","LAS HAMACAS","OLAS ALTAS","OLAS BAJAS","LIZCANO I","LIZCANO II","LOS ANGELES","MARIA PAZ",
                        "MESETAS DEL SANTUARIO","MINUTO DE DIOS","MIRAFLORES","MIRADOR","MIRADOR DEL KENNEDY","MIRAMAR","MODELO","MORRORICO","MUTUALIDAD","NAPOLES","NARIÑO",
                        "NAVAS","NORTE BAJO","OLAS II","OMAGA I","OMAGA II","PIO XII","PRIMAVERAL","PUERTO RICO","QUEBRADA LA IGLESIA I","REGADERO NORTE","RINCON DE LA PAZ",
                        "SAN CRISTOBAL","SAN RAFAEL","SAN MARTIN","SAN PEDRO CLAVER","SANTANDER","TEJAR NORTE","TRANSICION","TRANSICION I","TRANSICION II","TRANSICION III",
                        "TRANSICION IV","TRANSICION V","VILLA HELENA I","VILLA HELENA II","VILLA HELENA","VILLA ROSA","VILLA MARIA I","VILLA MARIA III", "VILLA MERCEDES",
                        "VILLA ROSA","VILLAS DE GIRARDOT", "ZARABANDA"]

El diccionario con las variables a cambiar:

In [16]:
barrios_cambiar = {
    "DIVINO NIÑO (COLORADOS)": "DIVINO NIÑO",
    "EL PABLON": "EL PABLÓN",
    "EL PABLON ( LA TORRE )": "EL PABLÓN",
    "EL PABLON ( VILLA LINA )": "EL PABLÓN",
    "EL PABLON ( VILLA PATRICIA )": "EL PABLÓN",
    "ESPERANZA I": "LA ESPERANZA I",
    "ESPERANZA II": "LA ESPERANZA II",
    "ESPERANZA III": "LA ESPERANZA III",
    "MIRADORES DEL KENNEDY": "MIRADOR DEL KENNEDY",
    "OMAGA (SECTOR I )": "OMAGA I",
    "OMAGA (SECTOR II )": "OMAGA II",
    "QUEBRADA LA IGLESIA": "QUEBRADA LA IGLESIA I",
    "TEJAR NORTE (SECTOR I )" : "TEJAR NORTE",
    "TEJAR NORTE (SECTOR II )" : "TEJAR NORTE",
    "VILLA MARIA" : "VILLA MARIA I",
    "VILLA MARIA III (BAJA)":"VILLA MARIA III",
    "VILLA ROSA (SECTOR II )":"VILLA ROSA",
    "VILLA ROSA (SECTOR III )":"VILLA ROSA",
    "INV. VILLAS DEL GIRARDOT" : "VILLAS DE GIRARDOT"
}

In [17]:
delitos["barrios_hecho"] = delitos["barrios_hecho"].replace(barrios_cambiar)

In [18]:
delitos = delitos[delitos["barrios_hecho"].isin(barrios_alertatemprana)]

In [19]:
delitos_por_barrios = delitos.groupby(["barrios_hecho"]).size().to_frame("quantity")

#delitos_por_barrios.to_csv("delitos_por_barrios.csv")

In [21]:
delitos_victima_mujer = delitos[delitos["genero"]=="FEMENINO"].groupby("barrios_hecho").size().to_frame("quantity")
delitos_victima_hombre = delitos[delitos["genero"]=="MASCULINO"].groupby("barrios_hecho").size().to_frame("quantity")

#delitos_victima_mujer.to_csv("delitos_victima_mujer.csv")
#delitos_victima_hombre.to_csv("delitos_victima_hombre.csv")

In [ ]:
delitos_vic_personamayor=delitos[delitos["curso_de_vida"]=="06. Persona Mayor"].groupby("barrios_hecho").size().to_frame("quantity")
delitos_vic_adulto=delitos[delitos["curso_de_vida"]=="05. Adultez"].groupby("barrios_hecho").size().to_frame("quantity")
delitos_vic_joven=delitos[delitos["curso_de_vida"]=="04. Jovenes"].groupby("barrios_hecho").size().to_frame("quantity")
delitos_vic_adolescente=delitos[delitos["curso_de_vida"]=="03. Adolescencia"].groupby("barrios_hecho").size().to_frame("quantity")
delitos_vic_infante=delitos[delitos["curso_de_vida"]=="02. Infancia"].groupby("barrios_hecho").size().to_frame("quantity")
delitos_vic_primerainfancia=delitos[delitos["curso_de_vida"]=="01. Primera infancia"].groupby("barrios_hecho").size().to_frame("quantity")

#delitos_vic_personamayor.to_csv("delitos_vic_personamayor.csv")
#delitos_vic_adulto.to_csv("delitos_vic_adulto.csv")
#delitos_vic_joven.to_csv("delitos_vic_joven.csv")
#delitos_vic_adolescente.to_csv("delitos_vic_adolescente.csv")
#delitos_vic_infante.to_csv("delitos_vic_infante.csv")
#delitos_vic_primerainfancia.to_csv("delitos_vic_primerainfancia.csv")


In [ ]:
delitos_no_fatales = delitos[delitos["clasificaciones delito"]=="Lesiones no fatales"].groupby("barrios_hecho").size().to_frame("quantity")
delitos_viol_sexual = delitos[delitos["clasificaciones delito"]=="Violencia sexual"].groupby("barrios_hecho").size().to_frame("quantity")
delitos_fatales = delitos[delitos["clasificaciones delito"]=="Lesiones fatales"].groupby("barrios_hecho").size().to_frame("quantity")

#delitos_no_fatales.to_csv("delitos_no_fatales.csv")
#delitos_viol_sexual.to_csv("delitos_viol_sexual.csv")
#delitos_fatales.to_csv("delitos_fatales.csv")

In [32]:
delitos["conducta"].value_counts()

HURTO A PERSONAS                                                                                   9944
LESIONES PERSONALES                                                                                8274
VIOLENCIA INTRAFAMILIAR                                                                            6596
LESIONES CULPOSAS ( EN ACCIDENTE DE TRANSITO )                                                     4756
HURTO A ENTIDADES COMERCIALES                                                                      1224
HURTO A RESIDENCIAS                                                                                1139
HURTO A MOTOCICLETAS                                                                               1094
HOMICIDIO                                                                                           739
ACTOS SEXUALES CON MENOR DE 14 AÑOS                                                                 607
ACCESO CARNAL ABUSIVO CON MENOR DE 14 AÑOS                      

In [33]:
delitos["armas_medios"].value_counts()

CONTUNDENTES                          11675
ARMA BLANCA / CORTOPUNZANTE            7180
SIN EMPLEO DE ARMAS                    6859
ARMA DE FUEGO                          3134
VEHICULO                               2933
MOTO                                   1900
LLAVE MAESTRA                           676
PALANCAS                                263
ESCOPOLAMINA                             84
PUNZANTES                                76
LLAMADA TELEFONICA                       61
CORTANTES                                35
PERRO                                    26
CORTOPUNZANTES                           19
DIRECTA                                  19
ACIDO                                    17
REDES SOCIALES                           13
MIXTA                                     9
LICOR ADULTERADO                          8
COMBUSTIBLE                               8
BICICLETA                                 8
GRANADA DE MANO                           8
AGUA CALIENTE                   

In [34]:
delitos["dia_semana"].value_counts()

7    5815
6    5530
1    5052
3    5035
2    4870
5    4802
4    4608
Name: dia_semana, dtype: int64

In [35]:
delitos["movil_agresor"].value_counts()

A PIE                    26640
CONDUCTOR MOTOCICLETA     3352
CONDUCTOR VEHICULO        2956
PASAJERO MOTOCICLETA      1402
CONDUCTOR TAXI             481
PASAJERO BUS               370
PASAJERO TAXI              250
BICICLETA                   89
PASAJERO VEHICULO           47
CONDUCTOR BUS               18
PASAJERO METRO               5
Name: movil_agresor, dtype: int64

In [26]:
#geojson = gpd.read_file("https://raw.githubusercontent.com/Joaron4/team211_datasets/main/Barrios-polygon.geojson", driver = "GeoJSON")